# E2E 테스트 – FAQ-only RAG PoC  

1. '.env' 로부터 OpenAI 키 로딩  
2. FAQ JSONL → FAISS 인덱스 생성  
3. LangGraph FAQ 파이프라인 로드  
4. 샘플 질의에 대한 응답 생성  
5. RAGAS 평가로 정확도·신뢰도 확인

In [ ]:
# 1. env & 라이브러리
from pathlib import Path
from dotenv import load_dotenv
import os, json, pprint, sys, pandas as pd

ROOT = Path.cwd().parent  # 노트북은 tests/ 하위에 있다고 가정
sys.path.append(str(ROOT))

load_dotenv(ROOT / ".env")
print(f"ROOT - {ROOT}")
print("환경 변수 로드 완료")


ROOT - c:\Users\insung\Finance_Agent
환경 변수 로드 완료


In [2]:
# 2. 인덱스 준비
from scripts.build_faq_index import DATA, OUT, emb, Document, FAISS

if OUT.exists():
    print("FAISS 인덱스 존재 → 로드")
    db = FAISS.load_local(
        folder_path=str(OUT.with_suffix("")),
        embeddings=emb,
        allow_dangerous_deserialization=True,
    )
else:
    # (인덱스가 없으면 빌드)
    with DATA.open(encoding="utf-8") as f:
        docs = []
        for line in f:
            row = json.loads(line)
            content = f"Q: {row['question']}\nA: {row['answer']}"
            docs.append(Document(page_content=content, metadata=row))
    db = FAISS.from_documents(docs, emb)
    db.save_local(str(OUT.with_suffix("")))
    print("FAISS index saved:", OUT)
    print("Docs:", db.index.ntotal)

FAISS index saved: C:\Users\insung\Finance_Agent\index\faq_faiss.index
FAISS index saved: C:\Users\insung\Finance_Agent\index\faq_faiss.index
Docs: 32


In [3]:
# 3. 파이프라인 로딩
from services.rag.faq_rag.faq_chain import graph

sample_q = "인터넷뱅킹에서 보안매체 없이 이체하려면 어떻게 하나요?"
result = graph.invoke({"question": sample_q})
pprint.pp(result)


{'question': '인터넷뱅킹에서 보안매체 없이 이체하려면 어떻게 하나요?',
 'context': 'Q: 인터넷뱅킹에서 보안매체와 인증서 없이 간편하게 이체하려면 어떻게 해야되나요?\n'
            'A: 간편이체 서비스를 가입 후 이용하려는 PC를 지정단말로 등록하면, 직접 설정한 간편이체 한도까지 '
            '보안매체(보안카드/OTP)와 인증서 입력 없이 이체할 수 있습니다.\n'
            '단, 간편이체 한도는 1일 최대 500만원 이내로 설정 가능하며, 전자금융 이체한도를 초과할 수 없습니다.\n'
            '\n'
            '간편이체 서비스는 즉시이체에만 적용되며, 예약이체 또는 지정한 간편이체한도 초과하여 이체, 미지정단말에서 이체 등의 '
            '거래 시에는 적용되지 않습니다. 또한 간편이체 서비스에 의한 거래임에도 안전한 거래로 인식되지 않은 경우 보안매체 '
            '입력을 요구 할 수 있습니다.\n'
            '\n'
            '▶ 간편이체 서비스 경로 안내 : 개인인터넷뱅킹 → 뱅킹관리 → 간편이체 서비스\n'
            '\n'
            '※ 우리WON뱅킹에서는 간편이체서비스가 제공되지 않습니다. 단, 우리WON인증서로 로그인 후 1천만원 이내 이체 '
            '시에는 보안매체 없이 우리WON인증서로 이체 가능하며, 소액이체(300만원 이하)인 경우에는 인증서 입력도 생략 '
            '가능합니다.\n'
            '\n'
            'Q: 인터넷뱅킹에서 인터넷/스마트뱅킹 이체한도 증액이 가능한가요?\n'
            'A: 인터넷뱅킹에서는 인터넷/스마트뱅킹 이체한도 조회 및 축소는 가능하나 증액은 불가합니다. 이체한도 증액은 '
            '우리WON뱅킹에서 비대면 실명확인 절차로 진행 할 수 있습니다.\n'
        

## 배치 테스트 & RAGAS 평가  
- 10개 샘플 질의를 선택해 파이프라인을 실행  
- answer relevancy / faithfulness / LLMContextPrecisionWithoutReference으로 지표 계산
- 레퍼런스가 없기 때문에 LLMContextPrecisionWithoutReference으로 답변과 컨텐스트 정보의 중복 정도로 precision 평가

In [ ]:
# 4. 배치 & RAGAS
from ragas import EvaluationDataset, SingleTurnSample, evaluate
from ragas.metrics import answer_relevancy, faithfulness, LLMContextPrecisionWithoutReference
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 1) 평가용 LLM/임베딩 (Ragas 래퍼로 감싸기)
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini", temperature=0))
evaluator_emb = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

# 2) 그래프 호출 → Ragas 샘플로 변환
SAMPLE_QUESTIONS = [
    "해외에서 인터넷뱅킹 쓰려면 출국 전 확인할 점?",
    "OTP 분실 시 거래 제한 해제 방법 알려줘",
    "인터넷뱅킹 이용자비밀번호 오류 해제 가능?",
    "간편이체 서비스 가입 절차가 궁금해",
    "지정 단말기에서만 접속하도록 설정할 수 있나요?",
]

samples = []
for q in SAMPLE_QUESTIONS:
    res = graph.invoke({"question": q})
    ans = res.get("answer", "")
    ctxs = res.get("contexts", [])
    if isinstance(ctxs, str):
        ctxs = [c for c in ctxs.split("\n\n") if c.strip()]

    samples.append(
        SingleTurnSample(
            user_input=q,
            response=ans,
            retrieved_contexts=ctxs,
        )
    )

dataset = EvaluationDataset(samples=samples)

# 3) 메트릭 설정 (레퍼런스 없이 가능한 조합)
metrics = [
    answer_relevancy,                                       # Q - A 관련성
    faithfulness,                                           # A가 컨텍스트에 근거?
    LLMContextPrecisionWithoutReference(llm=evaluator_llm)  # 컨텍스트가 Q와 관련?
    ]

# 4) 평가 실행
report = evaluate(
    dataset,
    metrics=metrics,
    llm=evaluator_llm,
    embeddings=evaluator_emb,
    show_progress=True,
    batch_size=8,
)

df_scores  = report.to_pandas()

df_overall = df_scores[[
    "answer_relevancy",
    "faithfulness",
    "llm_context_precision_without_reference"
]].mean()

print("=== Overall Metrics ===")
print(df_overall)

print("\n=== Sample-level Metrics (sorted by Faithfulness) ===")
print(df_scores.sort_values("faithfulness")[[
    "user_input",
    "retrieved_contexts",
    'response',
    "answer_relevancy",
    "faithfulness",
    "llm_context_precision_without_reference"
]])

Evaluating:   0%|          | 0/15 [00:00<?, ?it/s]

Batch 1/2:   0%|          | 0/8 [00:00<?, ?it/s]

=== Overall Metrics ===
answer_relevancy                           0.368960
faithfulness                               0.742857
llm_context_precision_without_reference    0.900000
dtype: float64

=== Sample-level Metrics (sorted by Faithfulness) ===
                   user_input  \
1    OTP 분실 시 거래 제한 해제 방법 알려줘   
2     인터넷뱅킹 이용자비밀번호 오류 해제 가능?   
3         간편이체 서비스 가입 절차가 궁금해   
0  해외에서 인터넷뱅킹 쓰려면 출국 전 확인할 점?   
4  지정 단말기에서만 접속하도록 설정할 수 있나요?   

                                  retrieved_contexts  \
1  [Q: 인터넷뱅킹을 오랫동안 이용하지 않아서 자금이체가 제한되었다고 나와요. 인터넷...   
2  [Q: 이용자비밀번호(로그인비밀번호) 2회 오류 상태인데, 인터넷뱅킹에서 재등록 할...   
3  [Q: 인터넷뱅킹에서 보안매체와 인증서 없이 간편하게 이체하려면 어떻게 해야되나요?...   
0  [Q: 해외에서 인터넷/스마트뱅킹을 이용하려면 출국하기 전에 확인할 사항이 있나요?...   
4  [Q: 지정된 기기에서만 인터넷/스마트뱅킹을 이용하고 싶어요.\nA: 단말기지정서비...   

                                            response  answer_relevancy  \
1  OTP 분실 시 거래 제한 해제 방법은 다음과 같습니다.\n\n1. 보안카드를 지참...          0.000000   
2  이용자비밀번호 오류 해제는 가능합니다. 기존 이용자비밀번호를 알고 있다면 인터넷뱅킹...          0.47

In [5]:
for i in range(len(df_scores)):
    print('\nuser_input:',df_scores['user_input'][i])
    print('retrieved_contexts:',df_scores['retrieved_contexts'][i])
    print('response:',df_scores['response'][i])
    print('='*120)


user_input: 해외에서 인터넷뱅킹 쓰려면 출국 전 확인할 점?
retrieved_contexts: ['Q: 해외에서 인터넷/스마트뱅킹을 이용하려면 출국하기 전에 확인할 사항이 있나요?\nA: 해외 출국 전 아래 사항을 확인 바랍니다.\n1) 해외IP차단서비스 해제 (필수)\n\n2) 단말기지정서비스 해제 또는 미지정기기 옵션 허용 권장\n\n3) OTP발생기(디지털OTP) 사용자는 보안카드 지참 권장\n\n- 고장, 분실 등 비상시 보안매체를 보안카드로 변경 가능(단, 전자뱅킹 이체한도 하향될 수 있음)\n\n4) 금융인증서/우리WON인증서 사용자는 공동인증서 추가 발급 권장\n\n5) 비밀번호 오류상태 사전 체크\n\n6) 전자뱅킹 이체한도 및 이용하려는 계좌 상태(한도제한 등) 확인\n\n7) 은행에 등록된 고객정보(성명, 연락처 등) 정비\n\n\n\n※ 국내통신망이 아닌 경우 이용이 불가할 수 있으며, 국내통신망을 이용하더라도 현지 통신망 상황에 따라 일부 제한될 수 있습니다.', "Q: [해외제신고] 해외에 거주하는 경우 인터넷/스마트뱅킹 비밀번호 오류해제 및 서비스 변경 등 업무처리 방법은?\nA: 해외 체류중인 고객님이 인터넷/스마트뱅킹 제신고 업무가 필요한 경우 아래 방법으로 처리 가능합니다.\n\n\n\n※ 인터넷/스마트뱅킹 제신고 업무 : 이용자비밀번호 3회 오류해제, OTP발생기/보안카드 비밀번호 오류해제, 이체한도 증액 등\n\n(단, 이용자비밀번호 3회 오류해제는 ＇2) 국내 대리인을 통한 제신고 방법'으로 진행 바람)\n\n\n\n1) 해외 우리은행 영업점을 통한 제신고 방법\n\n거주지역 해외영업점에 국내에서 파견된 본국 직원 근무여부를 확인하신 후 필요서류(*)를 지참하여 방문하시면, 본인확인 후 서류를 국내영업점(**)으로 송부하여 처리하므로 시간과 비용이 발생될 수 있습니다. 방문 전에 해외영업점과 국내영업점에 상담 후 진행을 부탁드립니다.\n\n(*) 필요서류 : 대한민국에서 발급한 유효한 신분증(주민등록증, 운전면허

- answer_relevancy ≥ 0.8 이면 FAQ 일치율 양호   낮으면 프롬프트/쿼리 확장/리트리버 k값 조정.

- faithfulness < 0.9 라면 Prompt 수정·retriever k값 조정 필요, ‘컨텍스트 내 근거 인용’ 지시 강화, 답변 길이 제한, 시스템 프롬프트 보강.

- context_precision 은 1.0에 가까울수록 불필요한 문맥이 적음. 임베딩 모델/토크나이저, 인덱스 파라미터(HNSW, k), 쿼리 재작성 확인.

 OOD(미지원 질문)용 거절 정확도 같은 보조 지표도 추가하면 좋을 듯.